### Import Libraries

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np

### Capture Video Stream

In [3]:
cap = cv2.VideoCapture('Video/face_track.mp4')

### Tale first frame of the video

In [4]:
ret, frame = cap.read()

### Set up the initial tracking window

In [6]:
face_casc=cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
face_rects=face_casc.detectMultiScale(frame)

### Covert the list to a Tuple

In [7]:
face_x,face_y,w,h = tuple(face_rects[0])
track_window=(face_x,face_y,w,h)

### Set Up the ROI for Tracking

In [8]:
roi = frame[face_y:face_y+h,
           face_x:face_x+w]

### HSV color maping

In [9]:
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

### Histogram to target on each frame for the meanshift calculation

In [11]:
roi_hist=cv2.calcHist([hsv_roi],
                     [0],
                     None,
                     [180],
                     [0,180])

### Normalize the histogram

In [13]:
cv2.normalize(roi_hist,
             roi_hist,
             0,
             255,
             cv2.NORM_MINMAX);


### Set the termination criteria

10 iterations or move 1 pt

In [14]:
term_crit=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

### It's a Kind of Magic

In [16]:
#While Loop
while(True):
    
    #capture video
    ret, frame=cap.read()
    
    #if statement
    if ret == True:
        
        #Frame in HSV
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        #calculate the base of ROI
        dest_roi = cv2.calcBackProject([hsv],
                                      [0],
                                      roi_hist,
                                      [0,100],
                                      1)
        
        #meanshift to get the new coordinates of the rectangle
        ret, track_window = cv2.meanShift(dest_roi,
                                         track_window,
                                         term_crit)
        
        #draw new rectangle on image
        x,y,w,h = track_window
        
        #open new window and display
        
        img2= cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),3)
        cv2.imshow('FireTracker',img2)
        
        #close window
        if cv2.waitKey(50)& 0xFF == 27:
            break
            
    else:
        break
        

#release and destroy
cap.release()
cv2.destroyAllWindows()